In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Dropout
import re

In [7]:
data = pd.read_csv('train.csv') # importing data

In [8]:
data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [9]:
data = data.dropna()
data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [10]:
indep_feature = data.drop('label', axis = 1) # independent variable

In [11]:
indep_feature

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [12]:
dep_feature = data.label # dependent Variable

In [13]:
dep_feature

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [14]:
# we need to reindex the data as we have removed the nan values 

messages = indep_feature.copy()
messages.reset_index(inplace = True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [15]:
ps = PorterStemmer()
corpus = []

In [16]:
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # to have only alphabats
    review = review.lower() # to have unique words we need to lower the words
    review = review.split() # split the words to perform steming etc
    
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [48]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [18]:
voc_size = 10000

In [26]:
one_hot_rep = [one_hot(words,voc_size) for words in corpus]
one_hot_rep

[[8780, 5308, 8362, 4623, 925, 5508, 1651, 6150, 8627, 8858],
 [8307, 710, 9542, 7165, 3234, 4067, 9686],
 [9568, 9716, 2525, 7327],
 [667, 6227, 2323, 7271, 3256, 6092],
 [2984, 3234, 7323, 8554, 4779, 6298, 3234, 5986, 9693, 2404],
 [1842,
  299,
  8749,
  2302,
  5913,
  1359,
  4626,
  3488,
  4485,
  5418,
  5429,
  9530,
  6124,
  8570,
  9686],
 [303, 6910, 2562, 918, 6658, 8297, 9124, 4440, 3250, 1135, 9176],
 [3834, 9069, 4268, 8456, 6587, 5715, 1359, 2361, 3250, 1135, 9176],
 [4919, 2965, 7252, 126, 935, 3286, 4352, 8231, 1359, 6002],
 [2631, 882, 1583, 1078, 2970, 7556, 4591, 1609],
 [1177, 2753, 9939, 7787, 6435, 8389, 1211, 9757, 1809, 6512, 3961],
 [7271, 3013, 925, 3286, 1359, 6587],
 [7181, 6666, 8917, 6112, 696, 21, 4031, 6079, 9954],
 [6231, 7048, 7534, 6873, 5379, 2221, 2741, 3250, 1135, 9176],
 [4309, 6102, 1195, 192, 7256, 3250, 1135, 9176],
 [6643, 7019, 3461, 5535, 9618, 5339, 6776, 6355, 5261, 6436],
 [1503, 3813, 710],
 [1458, 7093, 7964, 2495, 1359, 6285, 3567

In [20]:
sent_len = 20

In [27]:
sent_length=20
padding=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)
print(padding)

[[   0    0    0 ... 6150 8627 8858]
 [   0    0    0 ... 3234 4067 9686]
 [   0    0    0 ... 9716 2525 7327]
 ...
 [   0    0    0 ... 3250 1135 9176]
 [   0    0    0 ... 5753 6037 4844]
 [   0    0    0 ... 3830 7000 3987]]


In [28]:
len(padding)

18285

In [29]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
dep_feature.shape

(18285,)

In [35]:
X_final = np.array(padding)
y_final = np.array(dep_feature)

In [33]:
X_final.shape

(18285, 20)

In [34]:
y_final

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [37]:
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final, test_size = 0.33, random_state = 35)

In [38]:
model.fit(X_train,y_train,validation_data = (X_test,y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 11s 37ms/step - loss: 0.4715 - accuracy: 0.7667 - val_loss: 0.1839 - val_accuracy: 0.9225
Epoch 2/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1293 - accuracy: 0.9536 - val_loss: 0.2025 - val_accuracy: 0.9200
Epoch 3/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0852 - accuracy: 0.9733 - val_loss: 0.2141 - val_accuracy: 0.9221
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0502 - accuracy: 0.9838 - val_loss: 0.2389 - val_accuracy: 0.9168
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0296 - accuracy: 0.9927 - val_loss: 0.3147 - val_accuracy: 0.9026
Epoch 6/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0247 - accuracy: 0.9921 - val_loss: 0.3423 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0089 - accuracy: 0.9979 - val_loss: 0.5053 - val_accuracy: 0.907

In [42]:
y_pred = model.predict_classes(X_test)

C:\Users\USER\.conda\envs\carprediction\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [48]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
confusion_matrix(y_test,y_pred)

array([[3174,  233],
       [ 308, 2320]], dtype=int64)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9103562551781276